Cleaning the data.

In [21]:
# Import dependencies
from pathlib import Path
import pandas as pd

from pymongo import MongoClient
from pprint import pprint
import json

In [22]:
# Define the path to the file
flight_data = Path("Data/US Airline Flight Routes and Fares 1993-2024.csv")

# Read the data from the file
flight_data_df = pd.read_csv(flight_data)
flight_data_df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_22132\3616092142.py:5: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  flight_data_df = pd.read_csv(flight_data)


,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,...,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,Geocoded_City1,Geocoded_City2,tbl1apk
0,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,14112,ABE,...,81.43,G4,1.0000,81.43,G4,1.0000,81.43,NaN,NaN,202131013514112ABEPIE
1,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,15304,ABE,...,208.93,DL,0.4659,219.98,UA,0.1193,154.11,NaN,NaN,202131013515304ABETPA
2,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11259,ABQ,...,184.56,WN,0.9968,184.44,WN,0.9968,184.44,NaN,NaN,202131014011259ABQDAL
3,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11298,ABQ,...,182.64,AA,0.9774,183.09,AA,0.9774,183.09,NaN,NaN,202131014011298ABQDFW
4,Table1a,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",10140,14107,ABQ,...,177.11,WN,0.6061,184.49,AA,0.3939,165.77,NaN,NaN,202131014014107ABQPHX


In [23]:
# Drop the columns and rows that contain missing values
flight_data_df = flight_data_df.dropna()

In [24]:
flight_data_df.head()

,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,...,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,Geocoded_City1,Geocoded_City2,tbl1apk
5919,Table 1a,2010,1,34614,33195,"Salt Lake City, UT","Tampa, FL (Metropolitan Area)",14869,15304,SLC,...,226.59,DL,0.38,247.69,US,0.20,166.99,"Salt Lake City, UT\r\n(40.758478, -111.888142)","Tampa, FL (Metropolitan Area)\r\n(37.8606, -78...",201011486915304SLCTPA
5920,Table 1a,1998,4,30189,31703,"Colorado Springs, CO","New York City, NY (Metropolitan Area)",11109,12197,COS,...,280.39,UA,0.73,292.60,NW,0.24,248.27,"Colorado Springs, CO\r\n(38.835224, -104.819798)","New York City, NY (Metropolitan Area)\r\n(40.1...",199841110912197COSHPN
5921,Table 1a,1998,4,30198,30852,"Pittsburgh, PA","Washington, DC (Metropolitan Area)",14122,10821,PIT,...,239.12,US,0.93,245.70,CO,0.03,71.30,"Pittsburgh, PA\r\n(40.442169, -79.994945)","Washington, DC (Metropolitan Area)\r\n(38.8920...",199841412210821PITBWI
5922,Table 1a,2009,3,32211,32575,"Las Vegas, NV","Los Angeles, CA (Metropolitan Area)",12889,14908,LAS,...,111.74,WN,0.58,118.28,US,0.41,102.75,"Las Vegas, NV\r\n(36.169202, -115.140597)","Los Angeles, CA (Metropolitan Area)\r\n(34.052...",200931288914908LASSNA
5923,Table 1a,1993,4,30255,30852,"Huntsville, AL","Washington, DC (Metropolitan Area)",12217,12264,HSV,...,242.76,DL,0.48,275.50,UA,0.25,197.69,"Huntsville, AL\r\n(34.729538, -86.585283)","Washington, DC (Metropolitan Area)\r\n(38.8920...",199341221712264HSVIAD


In [25]:
# Display the data types of each column
flight_data_df.dtypes

tbl                object
Year                int64
quarter             int64
citymarketid_1      int64
citymarketid_2      int64
city1              object
city2              object
airportid_1         int64
airportid_2         int64
airport_1          object
airport_2          object
nsmiles             int64
passengers          int64
fare              float64
carrier_lg         object
large_ms          float64
fare_lg           float64
carrier_low        object
lf_ms             float64
fare_low          float64
Geocoded_City1     object
Geocoded_City2     object
tbl1apk            object
dtype: object

In [26]:
# Rename these columns: 
# citymarketid_1 to departure_city_id, citymarketid_2 to arrival_city_id, city1 to departure_city, city2 to arrival_city
# airportid_1 to departure_airport_id, airportid_2 to arrival_airport_id, airport_1 to departure_airport, airport_2 to arrival_airport
flight_data_df = flight_data_df.rename(columns={
    "citymarketid_1": "departure_city_id",
    "citymarketid_2": "arrival_city_id",
    "city1": "departure_city",
    "city2": "arrival_city",
    "airportid_1": "departure_airport_id",
    "airportid_2": "arrival_airport_id",
    "airport_1": "departure_airport",
    "airport_2": "arrival_airport",
})

# Display the first 5 rows of the updated DataFrame
flight_data_df.head()

,tbl,Year,quarter,departure_city_id,arrival_city_id,departure_city,arrival_city,departure_airport_id,arrival_airport_id,departure_airport,...,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,Geocoded_City1,Geocoded_City2,tbl1apk
5919,Table 1a,2010,1,34614,33195,"Salt Lake City, UT","Tampa, FL (Metropolitan Area)",14869,15304,SLC,...,226.59,DL,0.38,247.69,US,0.20,166.99,"Salt Lake City, UT\r\n(40.758478, -111.888142)","Tampa, FL (Metropolitan Area)\r\n(37.8606, -78...",201011486915304SLCTPA
5920,Table 1a,1998,4,30189,31703,"Colorado Springs, CO","New York City, NY (Metropolitan Area)",11109,12197,COS,...,280.39,UA,0.73,292.60,NW,0.24,248.27,"Colorado Springs, CO\r\n(38.835224, -104.819798)","New York City, NY (Metropolitan Area)\r\n(40.1...",199841110912197COSHPN
5921,Table 1a,1998,4,30198,30852,"Pittsburgh, PA","Washington, DC (Metropolitan Area)",14122,10821,PIT,...,239.12,US,0.93,245.70,CO,0.03,71.30,"Pittsburgh, PA\r\n(40.442169, -79.994945)","Washington, DC (Metropolitan Area)\r\n(38.8920...",199841412210821PITBWI
5922,Table 1a,2009,3,32211,32575,"Las Vegas, NV","Los Angeles, CA (Metropolitan Area)",12889,14908,LAS,...,111.74,WN,0.58,118.28,US,0.41,102.75,"Las Vegas, NV\r\n(36.169202, -115.140597)","Los Angeles, CA (Metropolitan Area)\r\n(34.052...",200931288914908LASSNA
5923,Table 1a,1993,4,30255,30852,"Huntsville, AL","Washington, DC (Metropolitan Area)",12217,12264,HSV,...,242.76,DL,0.48,275.50,UA,0.25,197.69,"Huntsville, AL\r\n(34.729538, -86.585283)","Washington, DC (Metropolitan Area)\r\n(38.8920...",199341221712264HSVIAD


In [27]:
# Extract the longitude and latitude from the 'Geocoded_City1' and 'Geocoded_City2' columns
# Use regular expressions

flight_data_df['departure_latitude'] = flight_data_df['Geocoded_City1'].str.extract(r'(\d+\.\d+),')
flight_data_df['departure_longitude'] = flight_data_df['Geocoded_City1'].str.extract(r',\s*(-?\d+\.\d+)')

flight_data_df['arrival_latitude'] = flight_data_df['Geocoded_City2'].str.extract(r'(\d+\.\d+),')
flight_data_df['arrival_longitude'] = flight_data_df['Geocoded_City2'].str.extract(r',\s*(-?\d+\.\d+)')

# Drop the 'Geocoded_City1' and 'Geocoded_City2' columns
flight_data_df = flight_data_df.drop(columns=['Geocoded_City1', 'Geocoded_City2'])

In [28]:
# Check the first 5 rows of the updated DataFrame
flight_data_df.head()

,tbl,Year,quarter,departure_city_id,arrival_city_id,departure_city,arrival_city,departure_airport_id,arrival_airport_id,departure_airport,...,large_ms,fare_lg,carrier_low,lf_ms,fare_low,tbl1apk,departure_latitude,departure_longitude,arrival_latitude,arrival_longitude
5919,Table 1a,2010,1,34614,33195,"Salt Lake City, UT","Tampa, FL (Metropolitan Area)",14869,15304,SLC,...,0.38,247.69,US,0.20,166.99,201011486915304SLCTPA,40.758478,-111.888142,37.8606,-78.804199
5920,Table 1a,1998,4,30189,31703,"Colorado Springs, CO","New York City, NY (Metropolitan Area)",11109,12197,COS,...,0.73,292.60,NW,0.24,248.27,199841110912197COSHPN,38.835224,-104.819798,40.123164,-75.333718
5921,Table 1a,1998,4,30198,30852,"Pittsburgh, PA","Washington, DC (Metropolitan Area)",14122,10821,PIT,...,0.93,245.70,CO,0.03,71.30,199841412210821PITBWI,40.442169,-79.994945,38.892062,-77.019912
5922,Table 1a,2009,3,32211,32575,"Las Vegas, NV","Los Angeles, CA (Metropolitan Area)",12889,14908,LAS,...,0.58,118.28,US,0.41,102.75,200931288914908LASSNA,36.169202,-115.140597,34.052238,-118.243344
5923,Table 1a,1993,4,30255,30852,"Huntsville, AL","Washington, DC (Metropolitan Area)",12217,12264,HSV,...,0.48,275.50,UA,0.25,197.69,199341221712264HSVIAD,34.729538,-86.585283,38.892062,-77.019912


In [29]:
# Reset the index of the DataFrame
flight_data_df = flight_data_df.reset_index(drop=True)

# Display the first 5 rows of the updated DataFrame
flight_data_df.head()

,tbl,Year,quarter,departure_city_id,arrival_city_id,departure_city,arrival_city,departure_airport_id,arrival_airport_id,departure_airport,...,large_ms,fare_lg,carrier_low,lf_ms,fare_low,tbl1apk,departure_latitude,departure_longitude,arrival_latitude,arrival_longitude
0,Table 1a,2010,1,34614,33195,"Salt Lake City, UT","Tampa, FL (Metropolitan Area)",14869,15304,SLC,...,0.38,247.69,US,0.20,166.99,201011486915304SLCTPA,40.758478,-111.888142,37.8606,-78.804199
1,Table 1a,1998,4,30189,31703,"Colorado Springs, CO","New York City, NY (Metropolitan Area)",11109,12197,COS,...,0.73,292.60,NW,0.24,248.27,199841110912197COSHPN,38.835224,-104.819798,40.123164,-75.333718
2,Table 1a,1998,4,30198,30852,"Pittsburgh, PA","Washington, DC (Metropolitan Area)",14122,10821,PIT,...,0.93,245.70,CO,0.03,71.30,199841412210821PITBWI,40.442169,-79.994945,38.892062,-77.019912
3,Table 1a,2009,3,32211,32575,"Las Vegas, NV","Los Angeles, CA (Metropolitan Area)",12889,14908,LAS,...,0.58,118.28,US,0.41,102.75,200931288914908LASSNA,36.169202,-115.140597,34.052238,-118.243344
4,Table 1a,1993,4,30255,30852,"Huntsville, AL","Washington, DC (Metropolitan Area)",12217,12264,HSV,...,0.48,275.50,UA,0.25,197.69,199341221712264HSVIAD,34.729538,-86.585283,38.892062,-77.019912


In [30]:
# Find all the unique values in the 'carrier_lg' column
flight_data_df['carrier_lg'].unique()

array(['DL', 'UA', 'US', 'WN', 'AA', 'HP', 'CO', 'B6', 'YX', 'AS', 'FL',
       'NW', 'NK', 'TW', 'RU', 'DH', 'F9', 'J7', 'TZ', 'JI', 'RP', 'P9',
       'U5', 'N7', 'G4', 'NJ', 'QQ', 'WV', 'VX', 'SY', 'KW', 'KP', 'ZW',
       'UK', 'W7', 'HQ', 'FF', 'TB', 'LC', 'YY', 'PA', 'YV', '5J', 'KN',
       '9K', 'E9', 'PN', 'BF', '9N', 'U2', 'OE', 'W9', 'ZV', 'RL', 'T3',
       'OP', 'OO', 'AQ', 'QX', 'OH', 'KS', '3M', 'XJ', 'ZA', 'SX'],
      dtype=object)

In [31]:
# Find all the unique values in the 'carrier_low' column
flight_data_df['carrier_low'].unique()

array(['US', 'NW', 'CO', 'UA', 'F9', 'AA', 'WN', 'DL', 'HP', 'FL', 'B6',
       'YX', 'NJ', 'AS', 'TW', 'BF', 'NK', 'RU', 'DH', 'SY', 'J7', 'JI',
       'TZ', 'RP', 'G4', 'P9', 'U5', 'VX', 'WV', 'QQ', 'KW', 'AQ', 'ZA',
       'KP', 'W9', 'UK', 'HQ', 'W7', 'OE', 'N7', 'KN', 'FF', 'EV', 'TB',
       'LC', 'ZW', 'YY', 'YV', 'E9', 'PN', '9N', 'OH', 'U2', '5J', 'ZV',
       '9K', 'C8', 'RL', 'T3', 'OO', 'N5', 'KS', 'HA', '3M', 'OP', 'XJ',
       'QX', 'SX', 'A7'], dtype=object)

In [32]:
# Convert the airline codes to airline names
# Create a dictionary with airline codes as keys and airline names as values
# You can find a list of these codes and their corresponding airline names in the Bureau of Transportation Statistics website
airlines = {
    "WN": "Southwest Airlines Co.",
    "DL": "Delta Air Lines Inc.",
    "AA": "American Airlines Inc.",
    "UA": "United Air Lines Inc.",
    "B6": "JetBlue Airways",
    "AS": "Alaska Airlines Inc.",
    "NK": "Spirit Air Lines",
    "G4": "Allegiant Air",
    "F9": "Frontier Airlines Inc.",
    "HA": "Hawaiian Airlines Inc.",
    "SY": "Sun Country Airlines d/b/a MN Airlines",
    "VX": "Virgin America",
    "US": "US Airways Group Inc.",
    "CO": "Continental Airlines Inc.",
    "HP": "America West Airlines Inc.",
    "NW": "Northwest Airlines Inc.",
    "FL": "AirTran Airways Corporation",
    "TZ": "ATA Airlines Inc.",
    "YX": "Midwest Airlines Inc.",
    "EV": "Atlantic Southeast Airlines",
    "MQ": "Envoy Air",
    "OH": "PSA Airlines",
    "OO": "SkyWest Airlines",
    "YV": "Mesa Airlines",
    "9E": "Endeavor Air",
    "AX": "Trans States Airlines",
    "CP": "Compass Airlines",
    "PT": "Piedmont Airlines",
    "ZW": "Air Wisconsin Airlines"
}

In [33]:
# Replace the airline codes with the airline names
flight_data_df['carrier_lg'] = flight_data_df['carrier_lg'].replace(airlines)
flight_data_df['carrier_low'] = flight_data_df['carrier_low'].replace(airlines)

# Display the first 30 rows of the updated DataFrame
flight_data_df[['carrier_lg', 'carrier_low']].head(30)

,carrier_lg,carrier_low
0,Delta Air Lines Inc.,US Airways Group Inc.
1,United Air Lines Inc.,Northwest Airlines Inc.
2,US Airways Group Inc.,Continental Airlines Inc.
3,Southwest Airlines Co.,US Airways Group Inc.
4,Delta Air Lines Inc.,United Air Lines Inc.
5,Southwest Airlines Co.,Frontier Airlines Inc.
6,American Airlines Inc.,American Airlines Inc.
7,Southwest Airlines Co.,Southwest Airlines Co.
8,US Airways Group Inc.,Continental Airlines Inc.
9,Delta Air Lines Inc.,Delta Air Lines Inc.


In [34]:
# Save the cleaned data to a new CSV file
output_file = Path("Data/CSV/cleaned_flight_data.csv")
flight_data_df.to_csv(output_file, index=False)

In [35]:
# Make a connection to the MongoDB server
mongo = MongoClient(port=27017)

In [36]:
# List the databases
print(mongo.list_database_names())

['admin', 'config', 'epa', 'flight_data', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [37]:
# Connect to the 'flight_data' database
db = mongo['flight_data']

Import the data from "US Airline Flight Routes and Fares 1993-2024.csv" using this command:

```shell
mongoimport --db flight_data --collection flights --drop --type csv --headerline --file "Data/CSV/cleaned_flight_data.csv"
```

In [38]:
# List the collections
print(db.list_collection_names())

['locations', 'flights']


In [39]:
# Return the first document in the 'flights' collection
pprint(db.flights.find_one())

{'Year': 2010,
 '_id': ObjectId('679c2bfb9f914d124e2d32f5'),
 'arrival_airport': 'TPA',
 'arrival_airport_id': 15304,
 'arrival_city': 'Tampa, FL (Metropolitan Area)',
 'arrival_city_id': 33195,
 'arrival_latitude': 37.8606,
 'arrival_longitude': -78.804199,
 'carrier_lg': 'Delta Air Lines Inc.',
 'carrier_low': 'US Airways Group Inc.',
 'departure_airport': 'SLC',
 'departure_airport_id': 14869,
 'departure_city': 'Salt Lake City, UT',
 'departure_city_id': 34614,
 'departure_latitude': 40.758478,
 'departure_longitude': -111.888142,
 'fare': 226.59,
 'fare_lg': 247.69,
 'fare_low': 166.99,
 'large_ms': 0.38,
 'lf_ms': 0.2,
 'nsmiles': 1887,
 'passengers': 200,
 'quarter': 1,
 'tbl': 'Table 1a',
 'tbl1apk': '201011486915304SLCTPA'}


In [ ]:
# Check if the 'locations' collection exists, if it does, drop it
# This is to ensure that an error is not raised when creating a new collection with the same name
# in case this file is run multiple times
if 'locations' in db.list_collection_names():
    db.drop_collection('locations')

# Add a new collection to the database using this document for reference
db.create_collection('locations')

# Iterate through each document in the 'flights' collection and add it to the 'locations' collection
for flight in db.flights.find():
    location_document = {
        "_id": flight["_id"],
        "arrival_airport": flight["arrival_airport"],
        "arrival_airport_id": flight["arrival_airport_id"],
        "arrival_city": flight["arrival_city"],
        "arrival_city_id": flight["arrival_city_id"],
        "arrival_latitude": flight["arrival_latitude"],
        "arrival_longitude": flight["arrival_longitude"],
        "departure_airport": flight["departure_airport"],
        "departure_airport_id": flight["departure_airport_id"],
        "departure_city": flight["departure_city"],
        "departure_city_id": flight["departure_city_id"],
        "departure_latitude": flight["departure_latitude"],
        "departure_longitude": flight["departure_longitude"]
    }
    db.locations.insert_one(location_document)

In [41]:
# Print the first document in the "locations" collection
pprint(db.locations.find_one())

{'_id': ObjectId('679c2bfb9f914d124e2d32f5'),
 'arrival_airport': 'TPA',
 'arrival_airport_id': 15304,
 'arrival_city': 'Tampa, FL (Metropolitan Area)',
 'arrival_city_id': 33195,
 'arrival_latitude': 37.8606,
 'arrival_longitude': -78.804199,
 'departure_airport': 'SLC',
 'departure_airport_id': 14869,
 'departure_city': 'Salt Lake City, UT',
 'departure_city_id': 34614,
 'departure_latitude': 40.758478,
 'departure_longitude': -111.888142}


In [42]:
from bson import ObjectId

# Write the "flights" collection to a json file
def convert_objectid(obj):
    if isinstance(obj, ObjectId):
        return str(obj)
    raise TypeError("Object of type %s is not JSON serializable" % type(obj).__name__)

with open('Data/JSON/flights.json', 'w') as f:
    json.dump(list(db.flights.find()), f, default=convert_objectid)

In [43]:
# Write the "locations" collection to a json file
with open('Data/JSON/locations.json', 'w') as f:
    json.dump(list(db.locations.find()), f, default=convert_objectid)

In [44]:
# Close the connection
mongo.close()